In [1]:
import streamlit as st
import pandas as pd
from datetime import datetime, timedelta,time
from streamlit_gsheets import GSheetsConnection
from auxiliar.auxiliar import *
from auxiliar.omie_aux import *

In [2]:
data_inicial = "2025-03-18"
data_final = "2025-03-18"

dados_crm_df = paste_billcharges_with_json(data_inicial,data_final)

2025-03-25 16:09:59.690 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-25 16:10:00.054 "runner.magic_enabled" is not a valid config option. If you previously had this config option set, it may have been removed.
2025-03-25 16:10:00.060 
  command:

    streamlit run /home/ciroshia/Downloads/github/gerador_de_notas_procorpo/.venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-03-25 16:10:00.062 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-25 16:10:00.063 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-25 16:10:00.472 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-25 16:10:00.474 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored w

In [3]:
dados_crm_df

,quote_id,billCharge_id,customer_id,customer_name,store_name,quote_status,paymentMethod_name,billcharge_paidAt,bill_installmentsQuantity,bill_amount,servicos_json,os_id,id_conta_corrente,dados_cliente,isPaid,Tipo de Pagamento,billcharge_dueAt,amount
0,437581,1191513,543746,Daniela Costa Silva,JARDINS,completed,Cartão de Crédito (Vindi),2025-03-18 15:40:55,1,200.00,"{""cDadosAdicItem"": ""Servi\u00e7o Prestado - 11...",437581-01,2.839998e+09,"{""razao_social"": ""Daniela Costa Silva"", ""nome_...",True,Pontual - Cartão,2025-03-17 00:00:00,200.00
1,437541,1191435,1027794,Solange Aparecida Gonçalves Teixeira,IPIRANGA,completed,Cartão de Crédito Vindi à Vista,2025-03-18 13:24:01,1,294.00,"{""cDadosAdicItem"": ""Servi\u00e7o Prestado - 11...",437541-01,2.839998e+09,"{""razao_social"": ""Solange Aparecida Gon\u00e7a...",True,Pontual - Cartão,2025-03-17 00:00:00,294.00
2,437526,1191409,458452,Renata Casemiro do Prado,TATUAPÉ,completed,PIX,2025-03-18 12:08:42,1,270.00,"{""cDadosAdicItem"": ""Servi\u00e7o Prestado - 11...",437526-00,2.839998e+09,"{""razao_social"": ""Renata Casemiro do Prado"", ""...",True,Pontual - Dinheiro,2025-03-15 00:00:00,270.00
3,437530,1191392,395011,Laiz Santos Rhemann Dias,SANTOS,completed,Cartão de Crédito,2025-03-18 11:44:45,10,170.00,"{""cDadosAdicItem"": ""Servi\u00e7o Prestado - 11...",437530-01,2.839998e+09,"{""razao_social"": ""Laiz Santos Rhemann Dias"", ""...",True,Pontual - Cartão,2025-12-15 00:00:00,170.00
4,437530,1191391,395011,Laiz Santos Rhemann Dias,SANTOS,completed,Cartão de Crédito,2025-03-18 11:44:45,10,170.00,"{""cDadosAdicItem"": ""Servi\u00e7o Prestado - 11...",437530-01,2.839998e+09,"{""razao_social"": ""Laiz Santos Rhemann Dias"", ""...",True,Pontual - Cartão,2025-11-15 00:00:00,170.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1282,367136,981772,564078,Maria das Graças da Silva,JARDINS,completed,Recorrente - Cartão de Crédito (Vindi),2025-03-18 16:14:14,12,224.17,"{""cDadosAdicItem"": ""Servi\u00e7o Prestado - 98...",367136,2.839998e+09,"{""razao_social"": ""Maria das Gra\u00e7as da Sil...",True,Recorrente,2025-03-17 00:00:00,224.17
1283,366707,980567,635486,Lilian Alves de Oliveira Moraes,SANTO AMARO,completed,Recorrente - Cartão de Crédito (Vindi),2025-03-18 17:42:24,12,788.33,"{""cDadosAdicItem"": ""Servi\u00e7o Prestado - 98...",366707,2.839998e+09,"{""razao_social"": ""Lilian Alves de Oliveira Mor...",True,Recorrente,2025-03-15 00:00:00,788.33
1284,366253,979488,810319,Myrtes Vieira Galdino,JARDINS,completed,Recorrente - Cartão de Crédito (Vindi),2025-03-18 16:10:54,12,215.50,"{""cDadosAdicItem"": ""Servi\u00e7o Prestado - 97...",366253,2.839998e+09,"{""razao_social"": ""Myrtes Vieira Galdino"", ""nom...",True,Recorrente,2025-03-12 00:00:00,215.50
1285,363187,971871,947225,Caroline Santiago Zanobini,TIJUCA,completed,Recorrente - Cartão de Crédito (Vindi),2025-03-18 11:43:30,12,283.33,"{""cDadosAdicItem"": ""Servi\u00e7o Prestado - 97...",363187,2.839998e+09,"{""razao_social"": ""Caroline Santiago Zanobini"",...",True,Recorrente,2025-02-28 00:00:00,283.33


In [4]:
dados_crm_df.shape

(1287, 18)